In [ ]:
from tkinter import *
from tkinter import filedialog
import os

class Application(Frame):

    def upload(self):
        self.filename = filedialog.askopenfilename()
        self.changelabel()

    def createWidgets(self):
        self.label = Label(root, text="Hello",wraplength=300,justify='left')
        frame = Frame(self, relief=RAISED, borderwidth=1)
        frame.pack(fill=BOTH, expand=True)

        browseButton = Button(self, text="Browse", background="light blue", fg="black",command = self.upload,height = 2, width = 7)
        okButton = Button(self, text="OK",height = 2, width = 7,background="light green", fg="black")
        browseButton.pack(side=RIGHT, padx=10, pady=10)
        okButton.pack(side=RIGHT, padx=10, pady=10)
        
        self.label.place(relx=0.05,rely=0.0)
        
        self.pack(fill=BOTH, expand=True)
        
    def changelabel(self):
        self.label.configure(text=self.filename)

    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.filename = None
        self.pack()
        self.createWidgets()
        self.changelabel()

root = Tk()
root.title("Pemetaan Emosi Lagu")
root.geometry('350x350')
app = Application(master=root)
app.mainloop()

In [ ]:
# image = tk.PhotoImage(file="myfile.gif")

In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from tkinter import *
from tkinter import filedialog

# #Load
def load(src):
    y, sr = librosa.load(src,mono=True,sr=22050)
    
    return y,sr

# #Convert
def convert(filename,path,y,sr):
    #Path
    dst = path+filename[:-4]+".wav"
    
    #COnver to WAV
    librosa.output.write_wav(dst, y, sr)

# #Get Feature

# #MFCC
def mfcc(y,sr,file):
    vector = list()
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13) #13 is default dimension 512 frame
    data = pd.DataFrame(mfcc)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,13,std))
    save = pd.DataFrame(vector)

    save.to_csv(path_to_fitur+file[:-4]+"/mfcc.csv", index=False, header=False)

# #TIMBRE // CENTROID // FLUX // ROLLOFF // ZERO CROSSING 
def timbre(y,sr,file):
    vector = list()
    cent = librosa.feature.spectral_centroid(y=y, sr=sr) #1 is default dimension 512 frame
    flux = librosa.onset.onset_strength(y=y, sr=sr) #1 is default dimension 512 frame
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) #1 is default dimension 512 frame
    zc = librosa.feature.zero_crossing_rate(y) #1 is default dimension 512 frame
    
    a = pd.DataFrame(cent) 
    b = pd.DataFrame(flux).T
    c = pd.DataFrame(rolloff)
    d = pd.DataFrame(zc)
    
    frame = [a,b,c,d]
    data = pd.concat(frame)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,4,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/timbre.csv", index=False, header=False)

# #SCF and SFM
def flatness(y,sr,file):
    vector = list()
    flatness = librosa.feature.spectral_flatness(y=y)
    data = pd.DataFrame(flatness)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,1,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/flatness.csv", index=False, header=False)
    
def crest(y,sr,file):
    peak = y.max()
    rms = librosa.feature.rmse(y=y)
    n = rms.size
    square = rms**2
    rms_v2 = math.sqrt((1/n)*(square.sum()))
    crest = [peak/rms_v2]
    
    save = pd.DataFrame(crest)
    
    save.to_csv(path_to_fitur+file[:-4]+"/crest.csv", index=False, header=False)

# #chroma
def chroma(y,sr,file):
    vector = list()
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=24) #24 is default dimension 512 frame
    data = pd.DataFrame(chroma)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,12,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/chroma.csv", index=False, header=False)

def fitur(file,y,sr):
    
    mfccs = mfcc(y,sr,file) #mfcc
    timbres = timbre(y,sr,file) #timbre
    flatnesss = flatness(y,sr,file) #sfm
    crests = crest(y,sr,file) #scf
    chromas = chroma(y,sr,file) #chroma
    
# #merge feature

def merge(case1,case2):
    case_a = pd.DataFrame(case1)
    case_b = pd.DataFrame(case2)
    
    merged = case_a.merge(case_b, how='outer', left_index=True, right_index=True)
    
    return merged

def flat(song,fitur):
    vector = list()
    
    data = pd.read_csv(path_to_fitur+str(song)+"/"+fitur+".csv",header=None)
    vector.append(np.squeeze(data.values,0))
        
#     result = np.array(vector)
#     mean = result.mean()
#     std = result.std()

#     normalized = (result-mean)/std

    return vector

def case1(file):
    fitur = 'mfcc'
    result = flat(file,fitur)
    df = pd.DataFrame(result)
    
    return df

def case2(file):
    case_1 = case1(file)
    fitur = 'timbre'
    case_2 = flat(file,fitur)
    result = merge(case_1,case_2)
    
    return result

def case3(file):
    case_2 = case2(file)
    fitur1 = 'crest'
    fitur2 = 'flatness'
    crest = flat(file,fitur1)
    flatness = flat(file,fitur2)
    results = merge(case_2,crest)
    result = merge(results,flatness)
    
    return result

def case4(file):
    case_3 = case3(file)
    fitur = 'chroma'
    chroma = flat(file,fitur)
    result = merge(case_3,chroma)
    
    return result

def norm(data):
    result = np.array(data)
    mean = result.mean()
    std = result.std()

    normalized = (result-mean)/std
    
    return normalized

# #Get emotion

def emotion(a,v):
    if ((v >= 0) & (a > 0)):
        cluster = 'cluster 1'
    if ((v < 0) & (a >= 0)):
        cluster = 'cluster 2'
    if ((v <= 0) & (a < 0)):
        cluster = 'cluster 3'
    if ((v > 0) & (a <= 0)):
        cluster = 'cluster 4'
        
    return cluster

def plot_scatter(a,v):
    plt.figure(figsize=(15, 15))
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    bbox=dict(boxstyle="round",
              ec=(1., 0.5, 0.5),
              fc=(1., 0.8, 0.8),
              alpha=0.6,
              color = 'yellow'
           )
    plt.text(0.6, 0.7, "cluster 1", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, 0.7, "cluster 2", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, -0.7, "cluster 3", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(0.6, -0.7, "cluster 4", size=30,
         ha="center", va="center",
         bbox=bbox
         )

    plt.scatter(v,a,color='b')
    plt.axhline(y=0,color='black', label ="X")
    plt.axvline(x=0,color='black', label ="Y")
    plt.show()
    
def proses(path):
    #Split path
    drive, path_and_file = os.path.splitdrive(path)
    paths, file = os.path.split(path_and_file)
    path_wav = drive+paths+'\wav\\'
    path_fitur = drive+paths+'\fitur\\'
    #load raw
    y, sr = load(path)

    #Convert Wav
    convert(file,path_wav,y,sr)

    #Load Wav
    y_wav, sr_wav = load(path_wav+file[:-4]+'.wav')

    #make dir
    try:  
        os.mkdir(path_fitur+file[:-4])
    except OSError:  
        print ("Creation of the directory %s failed" % path_fitur)

    # #Feature extraction
    fiture = fitur(file[:-4]+'.wav',y_wav,sr_wav)
    print("feature successfully")

    # #Flatten
    results = case2(file[:-4])
    print("flatten successfully")

    # #Norm
    normalized = norm(results)
    
    return normalized

class mclass:

    def __init__(self, window):

        self.window = window

        self.fig = Figure(figsize=(3.5, 3.5))
        self.a = self.fig.add_subplot(111)
        
        self.a.set_xlim([-1, 1])
        self.a.set_ylim([-1, 1])
        self.a.axhline(y=0,color='black', label ="X")
        self.a.axvline(x=0,color='black', label ="Y")
        
        bbox=dict(boxstyle="round",
              ec=(1., 0.5, 0.5),
              fc=(1., 0.8, 0.8),
              alpha=0.6,
              color = 'yellow'
               )
        self.a.text(0.6, 0.7, "cluster 1", size=10,
             ha="center", va="center",
             bbox=bbox
             )
        self.a.text(-0.6, 0.7, "cluster 2", size=10,
             ha="center", va="center",
             bbox=bbox
             )
        self.a.text(-0.6, -0.7, "cluster 3", size=10,
             ha="center", va="center",
             bbox=bbox
             )
        self.a.text(0.6, -0.7, "cluster 4", size=10,
             ha="center", va="center",
             bbox=bbox
             )

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.window)
        self.canvas.get_tk_widget().pack()
        self.txt = "Filename : "
        self.label = Label(window, text=self.txt,wraplength=300,justify='left')
        
        browseButton = Button(window, text="Browse", background="light blue", fg="black",command = self.upload,height = 2, width = 7)
        okButton = Button(window, text="OK",height = 2, width = 7,background="light green", fg="black")
        browseButton.pack(side=RIGHT, padx=10, pady=10)
        okButton.pack(side=RIGHT, padx=10, pady=10)
        
        self.label.place(relx=0.05,rely=0.0)

    def upload(self):
        self.filename = filedialog.askopenfilename()
        self.changelabel()
    
    def changelabel(self):
        self.label.configure(text=self.txt+self.filename)
        
    def read_inputs(self):
        x_input = self.box1.get()
        y_input = self.box2.get()

        def convert_to_float_list(x_in):
            x_input_list = x_in.split(',')
            x_floats = [float(x) for x in x_input_list]
            return x_floats

        x_array = np.array(convert_to_float_list(x_input))
        y_array = np.array(convert_to_float_list(y_input))

        return x_array,y_array

    def plot(self):
        self.a.cla()
        x,v = self.read_inputs()
        self.a.scatter(x, v, color='red')

        self.a.set_title ("Scatter Plot)", fontsize=16)
        self.a.set_ylabel("Y", fontsize=14)
        self.a.set_xlabel("X", fontsize=14)


        self.canvas.draw()

window = Tk()
start = mclass(window)
window.title("Pemetaan Emosi Lagu")
window.geometry('400x400')
window.mainloop()

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\USER\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\USER\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\USER\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\USER\Anaconda